In [17]:
import pygame
from pygame.locals import *
import glob
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt

In [18]:
class Config:
    IMG_PATH = "img/"
    SCROLL_SPEED = 4
    FPS = 60
    FLAP_COOLDOWN = 5
    SCREEN_WIDTH = 864
    SCREEN_HEIGHT = 936
    FLOOR_HEIGHT = 768
    GRAVITY = 0.75
    GRAVITY_LIMIT = 12
    JUMP_STRENGTH = 12
    PIPE_GAP = 150
    PIPE_FREQUENCY = 1500 # in milliseconds

#### Sprites

In [19]:
from pygame.sprite import Sprite, Group

class Bird(Sprite):
    def __init__(self, x, y):
        Sprite.__init__(self) # this must be done in this format
        self.images = []
        self.image_index = 0
        self.counter = 0
        for num in range(1, 4):
            self.images.append(pygame.image.load(f"{Config.IMG_PATH}/bird{num}.png"))
        self.image = self.images[self.image_index]
        
        # position of bird sprite
        self.rect = self.image.get_rect()
        self.rect.center = [x,y]
        
        # physics variables
        self.velocity = 0
        self.unique_jump = True
    
    def _apply_gravity(self):
        self.velocity = Config.GRAVITY_LIMIT if self.velocity > Config.GRAVITY_LIMIT else self.velocity+Config.GRAVITY
        if self.rect.bottom < 768 and self.rect.top >= 0:
            self.rect.y += int(self.velocity)

        if self.rect.top < 0:
            self.rect.top = 0
            self.velocity = 0
    
    def update(self):
        if not GameInstance.game_over:
            if GameInstance.game_started:
                self._apply_gravity()

            # jump handling
            if pygame.key.get_pressed()[pygame.K_SPACE] and self.unique_jump:
                self.velocity -= Config.JUMP_STRENGTH
                self.unique_jump = False
            if not pygame.key.get_pressed()[pygame.K_SPACE] and not self.unique_jump:
                self.unique_jump = True

            # handle the animation of the bird
            self.counter += 1
            if self.counter >= Config.FLAP_COOLDOWN:
                self.counter = 0
                self.image_index += 1

            if self.image_index >= len(self.images):
                self.image_index = 0

            # set image and rotation
            self.image = pygame.transform.rotate(self.images[self.image_index], -4*self.velocity)
        else: 
            self.image = pygame.transform.rotate(self.images[self.image_index], -90)
            self._apply_gravity()
            
    def check_collision(self, pipe):
        PIPE_RECT = pipe.rect
        has_collided = self.rect.colliderect(PIPE_RECT)
        if has_collided:
            return True
        return False

In [20]:
class Button():
    def __init__(self, x, y, image):
        self.image = image
        self.rect = self.image.get_rect()
        self.rect.topleft = (x,y)
    def draw(self, screen):
        screen.blit(self.image, (self.rect.x, self.rect.y))
    
    def was_clicked(self):
        action = False
        # get mouse postion
        pos = pygame.mouse.get_pos()
        if self.rect.collidepoint(pos) and pygame.mouse.get_pressed()[0]:
            action = True
        return action    

In [21]:
from pygame.sprite import Sprite, Group

class Pipe(Sprite):
    def __init__(self, x, y, bottom_pipe=True):
        Sprite.__init__(self) # this must be done in this format
        self.image = pygame.image.load(f"{Config.IMG_PATH}/pipe.png")
        self.is_bottom_pipe = bottom_pipe
        if bottom_pipe:
            self.rect = self.image.get_rect()
            self.rect.topleft = [x,y+int(Config.PIPE_GAP/2)]
        else:
            # given the pipe is coming from the top, we need to flips the original image on the y axis
            self.image = pygame.transform.flip(self.image, False, True)
            self.rect = self.image.get_rect()
            self.rect.bottomleft = [x,y-int(Config.PIPE_GAP/2)]
    
    def update(self,):
        if not GameInstance.game_over:
            self.rect.x -= Config.SCROLL_SPEED
            if self.rect.right < 0:
                self.kill() # reduce buffer holding pipes by deleting them once they're off screen

#### Helper Functions that are Static 
- No need to change these even after changes

In [22]:
def draw_text(screen, text, font, text_color, x, y):
    img = font.render(text, True, text_color)
    screen.blit(img, (x,y))

def draw_and_update_sprites(screen, sprite_groups):
    for sprite_group in sprite_groups:
        sprite_group.draw(screen)
        sprite_group.update()
        
def update_score(bird_group, pipe_group):
    if pipe_group:
        closest_bird = bird_group.sprites()[0]
        closest_pipe_right = pipe_group.sprites()[0].rect.right
        bird_left = closest_bird.rect.left
        
        if bird_left > closest_pipe_right and not GameInstance.pipe_passed:
            GameInstance.score += 1
            GameInstance.pipe_passed = True
        
        if bird_left <= closest_pipe_right and GameInstance.pipe_passed:
            GameInstance.pipe_passed = False

#### Helper Functions that need Changes after adding NEAT 

In [23]:
def detect_and_handle_collisons(bird_group, pipe_group):
    closest_bird = bird_group.sprites()[0]
    
    # print out velocity, and distance to top and bottom of pipe
    def distance(bird, pipe):
        bird_x, bird_y = bird.rect.center
        if pipe.is_bottom_pipe:
            pipe_x, pipe_y = pipe.rect.topleft
        else:
            pipe_x, pipe_y = pipe.rect.bottomleft
        return np.sqrt((bird_x - pipe_x)**2 + (bird_y - pipe_y)**2)
    
    if closest_bird.rect.bottom > Config.FLOOR_HEIGHT:
        GameInstance.game_over = True
        GameInstance.game_started = False
    
    if pipe_group:
        bottom_closest_pipe = pipe_group.sprites()[0]
        top_closest_pipe = pipe_group.sprites()[1]
        if not GameInstance.game_over:
            print(closest_bird.velocity, distance(closest_bird, bottom_closest_pipe), distance(closest_bird, top_closest_pipe))
        if closest_bird.check_collision(bottom_closest_pipe) or closest_bird.check_collision(top_closest_pipe):
            GameInstance.game_over = True
            
def reset_game(pipe_group, bird_group):
    # reset pipes
    pipe_group.empty()
    # reset birds
    bird_group.empty()    
    flappy_bird = Bird(100, int(Config.SCREEN_HEIGHT/2))
    bird_group.add(flappy_bird)

In [24]:
# define game variables
class GameInstance:
    ground_scroll = 0
    game_started = False
    game_over = False
    last_pipe_generation = 0
    score = 0
    pipe_passed = False

pygame.init()

# configure clock and framerate
clock = pygame.time.Clock()

# define font
font = pygame.font.SysFont("Bauhaus 93", 60)
white = (255,255,255)

# import images
background = pygame.image.load(f"{Config.IMG_PATH}/bg.png")
ground = pygame.image.load(f"{Config.IMG_PATH}/ground.png")
button_img = pygame.image.load(f"{Config.IMG_PATH}/restart.png")

button = Button(Config.SCREEN_WIDTH//2-50, Config.SCREEN_HEIGHT//2-100, button_img)

# add bird sprites - init_game
bird_group = Group()
flappy_bird = Bird(100, int(Config.SCREEN_HEIGHT/2))
bird_group.add(flappy_bird)

# add pipe sprites
pipe_group = Group()

# set screen size and display window
screen = pygame.display.set_mode((Config.SCREEN_WIDTH, Config.SCREEN_HEIGHT))
pygame.display.set_caption("Flappy Bird Demo")

# game loop
run = True
while run:
    
    clock.tick(Config.FPS)
    
    # draw background
    screen.blit(background, (0,0))
    
    # draw and update sprites
    draw_and_update_sprites(screen, [bird_group, pipe_group])
    
    # collision checks - need to change later
    detect_and_handle_collisons(bird_group, pipe_group)
            
    # score checking
    update_score(bird_group, pipe_group)
    draw_text(screen, str(GameInstance.score), font, white, int(Config.SCREEN_WIDTH/2), 20)
    
    # draw and scroll the ground
    screen.blit(ground, (GameInstance.ground_scroll, Config.FLOOR_HEIGHT))
    if not GameInstance.game_over and GameInstance.game_started:
        time_now = pygame.time.get_ticks()
        if time_now - GameInstance.last_pipe_generation > Config.PIPE_FREQUENCY:
            pipe_height = random.randint(int(Config.PIPE_GAP/2)+100, int(Config.FLOOR_HEIGHT-Config.PIPE_GAP/2-100))
            bottom_pipe = Pipe(Config.SCREEN_WIDTH, pipe_height, bottom_pipe=True)
            top_pipe = Pipe(Config.SCREEN_WIDTH, pipe_height, bottom_pipe=False)
            pipe_group.add(bottom_pipe)
            pipe_group.add(top_pipe)
            GameInstance.last_pipe_generation = time_now
        GameInstance.ground_scroll = 0 if abs(GameInstance.ground_scroll) > 35 else (GameInstance.ground_scroll - Config.SCROLL_SPEED)
    
    # event handling
    if GameInstance.game_over:
        button.draw(screen)
        if button.was_clicked():
            # reset game:
            GameInstance.game_over = False
            GameInstance.game_started = False
            GameInstance.score = 0
            reset_game(pipe_group, bird_group)
    
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE and not GameInstance.game_started:
                GameInstance.game_started = True
        
        if event.type == pygame.QUIT:
            run = False   
    pygame.display.update()
pygame.quit()

1.5 770.0493490679672 807.8836549899992
2.25 766.4280788175756 804.8055665811463
3.0 762.9842724460315 802.0879004198979
3.75 759.557766071811 799.3922691645198
4.5 756.3279711871035 797.0771606312653
5.25 753.3133478174935 795.1609899888198
6.0 750.5364747965284 793.665546688276
-5.25 747.8188283267546 792.2329202955403
-4.5 743.0915152254129 787.0101651185962
-3.75 738.5743022878605 782.1713367287247
-3.0 734.0633487649415 777.3345483123724
-2.25 729.7513275082135 772.8757985601568
-1.5 725.6335163152264 768.7938605374005
-0.75 721.7090826641993 765.0908442792921
0.0 717.7854832747734 761.3908326214599
0.75 713.8627319029898 757.6938695805846
1.5 710.1387188430159 754.3851801301507
-9.75 706.6208318468965 751.473885108458
-9.0 700.9279563550023 744.3117626371358
-8.25 695.5084471090197 737.5852493102069
-7.5 690.3339771443964 731.2735466294401
-6.75 685.3794569433782 725.3585320377779
-6.0 680.4652819946069 719.467163948432
-5.25 675.74255452798 713.9523793643383
-4.5 671.19296778199

#### TODO: 
- Clean up collision detection
- Clean up Pipe Life cycle
- Assign Singletons to a class
- Clean up event handling
- Clean up game over and game start
- Add init_game

### BIRD AI

### GENETIC ALGO

In [15]:
def test():
    return True if 0 else False

In [16]:
test()

False